In [1]:
import $cp.`/container/target/dependency/*`

import $cp.$

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@687b3912
import spark.implicits._

In [3]:
import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udaf

case class IntSumBuffer(var sum: Long = 0L)

object SumAll extends Aggregator[Int, IntSumBuffer, Long] {
  def zero = IntSumBuffer(0L)
  def reduce(buffer: IntSumBuffer, value: Int): IntSumBuffer = {
    buffer.sum += value
    buffer
  }
  def merge(b1: IntSumBuffer, b2: IntSumBuffer): IntSumBuffer = {
    b1.sum += b2.sum
    b1
  }
  def finish(reduction: IntSumBuffer): Long = reduction.sum
  def bufferEncoder = Encoders.product[IntSumBuffer]
  def outputEncoder = Encoders.scalaLong
}

val sumAll = udaf(SumAll)


import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udaf
defined class IntSumBuffer
defined object SumAll
sumAll: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedAggregator(
  aggregator = ammonite.$sess.cmd3$Helper$SumAll$@53ab4d63,
  inputEncoder = ExpressionEncoder(
    objSerializer = BoundReference(
      ordinal = 0,
      dataType = IntegerType,
      nullable = false
    ),
    objDeserializer = AssertNotNull(
      child = UpCast(
        child = GetColumnByOrdinal(ordinal = 0, dataType = IntegerType),
        target = IntegerType,
        walkedTypePath = List("- root class: \"int\"")
      ),
      walkedTypePath = List("- root class: \"int\"")
    ),
    clsTag = Int
  ),
  name = None,
  nullable = true,
  deterministic = true
)

In [4]:
val df = Seq(1, 2, 3, 4, 5).toDF("value")
df.select(sumAll($"value").as("total_sum")).show()

SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.


+---------+
|total_sum|
+---------+
|       15|
+---------+



df: org.apache.spark.sql.package.DataFrame = [value: int]